In [2]:
import requests
import json
import datetime
import re
import pymysql
import random


def clear_html_re(src_html):
    '''
    正则清除HTML标签
    :param src_html:原文本
    :return: 清除后的文本
    '''
    content = re.sub(r"</?(.+?)>", "", src_html) # 去除标签
    # content = re.sub(r"&nbsp;", "", content)
    dst_html = re.sub(r"\s+", "", content)  # 去除空白字符
    return dst_html


def get_UA():
    user_agents = [
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
        "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
        "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
        "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
        "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
        "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
        "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
        "Mozilla/5.0 (iPad; CPU OS 8_1_2 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) CriOS/47.0.2526.107 Mobile/12B440 Safari/600.1.4"

    ]
    user_agent = random.choice(user_agents)
    return user_agent



In [1]:
# 疯抢首页
url_vip_fengqiang = "https://mst.vip.com/Special/getBrandData"

headers={
    'Cookie': "mst_csrf_key=c9eed823be45f45d9b46f7fa7becdc3a; mars_pid=0; vip_wh=VIP_SH; vip_address=%257B%2522pid%2522%253A%2522103101%2522%252C%2522pname%2522%253A%2522%255Cu4e0a%255Cu6d77%255Cu5e02%2522%252C%2522cid%2522%253A%2522103101101%2522%252C%2522cname%2522%253A%2522%255Cu4e0a%255Cu6d77%255Cu5e02%2522%257D; vip_province=103101; vip_province_name=%E4%B8%8A%E6%B5%B7%E5%B8%82; vip_city_name=%E4%B8%8A%E6%B5%B7%E5%B8%82; vip_city_code=103101101; _smt_uid=5d0894a6.53e611f7; vip_ipver=31; VipUINFO=luc%3Aa%7Csuc%3Aa%7Cbct%3Ac_new%7Chct%3Ac_new%7Cbdts%3A0%7Cbcts%3A0%7Ckfts%3A0%7Cc10%3A0%7Crcabt%3A0%7Cp2%3A0%7Cp3%3A1%7Cp4%3A0%7Cp5%3A1; mars_sid=6906c7b410ecd035d1c06f28b825ee5a; m_vip_province=103101; _jzqco=%7C%7C%7C%7C%7C1.1279841833.1560843430982.1560864656734.1560864671263.1560864656734.1560864671263.0.0.0.11.11; mars_cid=1560843113101_98bb65dc93d822fde2b29e9dfecbc2df; mst_consumer=A1",
    "user-agent": get_UA()}


# 记录疯抢的场次数据
fengqiang_list = []
for offset in range(0, 500, 30):
    data_fengqiang = {
    "page_id": "7671992",
    "floor": "1f",
    "limit": "30",
    "offset": offset,
    "client": "vipcms",
    "is_personality": "true",
    "plugin_id": "136612969"
    }
    
    res=requests.get(url=url_vip_fengqiang, params=data_fengqiang, headers=headers).json()
    print(res.get("data"))
    if len(res.get("data")) == 0:
        break
    
    fengqiang_list.append(res)



NameError: name 'get_UA' is not defined

In [3]:
# 疯抢进入场次，插入每场的item
for fengqiang in fengqiang_list:
    data = fengqiang.get("data")
    value1_list = []
    for item in data:
        yesterday = datetime.date.today() + datetime.timedelta(days = -1)    # 昨天日期
        today = datetime.date.today()
        tomorrow = datetime.date.today() + datetime.timedelta(days = 1) 

        count_date = today
        item_id = item.get("id")
        list_vip_url = "https://list.vip.com/%s.html" % item_id
        list_vip_pic = item.get("mobile_image_one")
        list_vip_detail = item.get("name")
        fq_discount_range = clear_html_re(item.get("discount"))
        list_logo = item.get("logo")
        status = "已疯抢"
        print(item_id)
        
        #进入场次首页拿取商品的主题和尺寸
        # 拿取上面的item_id, 请求接口的每场疯抢的category
        url_category = "https://list.vip.com/api-ajax.php"

        data = {
            "callback": "getCategoryListCB",
            "ff": "brand",
            "getPart": "getCategoryList",
            "r": item_id,
            }


        
        res=requests.get(url=url_category, params=data, headers=headers)
        response_text = res.text.replace("getCategoryListCB(", "").replace(")", "")

        response_dict = json.loads(response_text, encoding="utf-8")

#         print(response_dict)
        # 主题
        categoryList = response_dict.get("data").get("getCategoryList").get("category")
        items_classfys = []
        for classify in categoryList:
            cate_name = classify.get("cate_name", "")
            total = classify.get("total", "")
            items_classfys.append("%s(%s)" % (cate_name, total))
        fq_classify = ','.join(items_classfys)
        

        # size
        fq_sizes = response_dict.get("data", {}).get("getCategoryList", {}).get("property", [])
        for fq_s in fq_sizes:
            if fq_s.get("name", "") == "尺码":
                size_lists = fq_s.get("list", [])
                break

        sizes = []
        for size in size_lists:
            sizes.append(size.get("name"))
        fq_size = ",".join(sizes)
        
        value = (count_date,item_id,list_vip_url,list_vip_pic,list_vip_detail,fq_discount_range,list_logo,fq_classify, fq_size, status)
        value1_list.append(value)


NameError: name 'fengqiang_list' is not defined

In [2]:
# 疯抢进入场次，插入每场的item
for fengqiang in fengqiang_list[2:]:
    data = fengqiang.get("data")
    print("fengqiang_list.index(fengqiang) =", fengqiang_list.index(fengqiang))
    value1_list = []
    for item in data[7:]:
        yesterday = datetime.date.today() + datetime.timedelta(days = -1)    # 昨天日期
        today = datetime.date.today()
        tomorrow = datetime.date.today() + datetime.timedelta(days = 1) 
        print("data.index(item) =", data.index(item))
        count_date = today
        item_id = item.get("id")
        print("[item_id] =", item_id)
        
        # 拿取上面的item_id, 请求接口的每场疯抢的category
        url_category = "https://list.vip.com/api-ajax.php"
        
        data_category = {
            "callback": "getCategoryListCB",
            "ff": "brand",
            "getPart": "getCategoryList",
            "r": item_id,
            }



        res=requests.get(url=url_category, params=data_category, headers=headers)
        response_text = res.text.replace("getCategoryListCB(", "").replace(")", "")
#         print(response_text)
        response_dict = json.loads(response_text, encoding="utf-8")
#         print(response_dict)
#         if not response_dict.get("data"):
            
        # print(response_dict)
        # 主题
        categoryList = response_dict.get("data").get("getCategoryList").get("category")

#         print(categoryList)

        for category in categoryList:
            cate_id = category.get("cate_id")
            cate_name = category.get("cate_name")
            total = category.get("total")
            print("categoryList.index(category) =", categoryList.index(category))
            print("cate_name", cate_name)
#             print("##" * 40)
#             print("[cate_id] =",cate_id)
#             print("##" * 40)

            # 根据category找每个商品id
            url_category_product_id = "https://list.vip.com/api-ajax.php"
            data_category_product_id = {
                "callback": "getMerchandiseIds",
                "getPart": "getMerchandiseRankList",
                "fromIndex": "0",
                "mInfoNum": "0",
                "batchSize": "200",
                "r": item_id,
                "q": "|%s||0|0|1" % cate_id,
                "preview": "0",
            }
            res=requests.get(url=url_category_product_id, params=data_category_product_id, headers=headers)
            res_product_dict = json.loads(res.text.replace("getMerchandiseIds(", "").replace(")", ""),encoding="utf-8")
#             print("##" * 40)
#             print("[res_product_dict] =", res_product_dict)
#             print("##" * 40)
            product_ids = res_product_dict.get("data").get("getMerchandiseRankList").get("products")
#             print(product_ids)
            len_product_id = len(product_ids)
            start_ind = 0

            for start_ind in range(0, len_product_id, 49):
                print("##" * 40)
                print("[start_ind] =", start_ind)
                print("###" * 40 )
                if len_product_id - start_ind < 49:
                    products_ids_str = ",".join(product_ids[start_ind:len_product_id - 1])
                else:
                    products_ids_str = ",".join(product_ids[start_ind:start_ind+49])

                # 根据商品id去找商品
                url_goods_id = "https://list.vip.com/api-ajax.php"
                data_goods_id = {
                    "callback": "getMerchandiseDroplets3",
                    "getPart": "getMerchandiseInfoList",
                    "r": item_id,
                    "preview": "0",
                    "functions": "banInfo",
                    "productIds": products_ids_str
                }

                res=requests.get(url=url_goods_id, params=data_goods_id, headers=headers)
#                 print(res.text)
                res_goods = json.loads(res.text.replace("getMerchandiseDroplets3(", "").replace(")", ""),encoding="utf-8")


                value1_list = []
                # 获取商品列表
                merchandiseinfolist = res_goods.get("data").get("getMerchandiseInfoList").get("merchandiseInfoList")
#                 print("[merchandiseinfolist] =", merchandiseinfolist)
                for merchandiseinfo in merchandiseinfolist:
#                     print(merchandiseinfo)
                    goods_id = merchandiseinfo.get("productId", "")
                    goods_pic = merchandiseinfo.get("smallImage", "")
                    goods_detail_url = merchandiseinfo.get("detailUrl", "")
                    fq_price = merchandiseinfo.get("vipshopPrice", "")
                    origin_price = merchandiseinfo.get("marketPrice", "")
                    goods_discount = merchandiseinfo.get("vipDiscount", "")
                    goods_detail = merchandiseinfo.get("productName", "")
#                     
                    value = (count_date,item_id,goods_id,cate_name, goods_pic,goods_detail_url,fq_price,origin_price,goods_discount,goods_detail)
                    value1_list.append(value)


#                 sql = """INSERT INTO ds_vip_fq_index_goods_d(count_date,item_id,goods_id,category, goods_pic,goods_detail_url,fq_price,origin_price,goods_discount,goods_detail)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

#                 try:
#                     cursor.executemany(sql, value1_list)
#                 except Exception as e:
#                     print(e)

#                 db.commit()


NameError: name 'fengqiang_list' is not defined